# Amazon Customer Reviews Task

### You will work with Amazon Customer Reviews, writing labeling functions that classify them as positive (1) or negative (0) sentiment. 

First, import necessary libraries:

In [ ]:
from data.preparer import load_amazon_dataset
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import PandasLFApplier
from analyzer import train_model
import re
import pandas as pd
pd.set_option('display.max_colwidth', -1)

## The Data

The reviews are from [Amazon](https://s3.amazonaws.com/amazon-reviews-pds/readme.html).

For simplicity, only 1 star and 5 star reviews are included.

In [ ]:
DELIMITER = "#"
df_train, df_dev, df_valid, df_test = load_amazon_dataset(delimiter=DELIMITER)
print("{} training examples".format(len(df_train)))
print("{} development examples".format(len(df_dev)))
print("{} validation examples".format(len(df_valid)))
print("{} test examples".format(len(df_test)))

Define the labels for this task:

In [ ]:
ABSTAIN = -1
NEGATIVE = 0
POSITIVE = 1

In [ ]:
print("Some labeled examples: ")
display(df_dev[df_dev.label==NEGATIVE].sample(5))
display(df_dev[df_dev.label==POSITIVE].sample(5))

## Labeling Instructions

All reviews were submitted with either 1 star (negative) or 5 star (positive) ratings. 

Your task is to __create 10 labeling functions__ that take the text of the review as input, and output either a NEGATIVE or a POSITIVE or an ABSTAIN label. Try to write them as quickly and accurately as possible.

You may consult the internet at any time.

## Writing Labeling Functions
Time to write some labeling functions!

In [ ]:
@labeling_function()
def lf0(x):
    return ABSTAIN

In [ ]:
@labeling_function()
def lf1(x):
    return ABSTAIN

In [ ]:
@labeling_function()
def lf2(x):
    return ABSTAIN

In [ ]:
@labeling_function()
def lf3(x):
    return ABSTAIN

In [ ]:
@labeling_function()
def lf4(x):
    return ABSTAIN

In [ ]:
@labeling_function()
def lf5(x):
    return ABSTAIN

In [ ]:
@labeling_function()
def lf6(x):
    return ABSTAIN

In [ ]:
@labeling_function()
def lf7(x):
    return ABSTAIN

In [ ]:
@labeling_function()
def lf8(x):
    return ABSTAIN

In [ ]:
@labeling_function()
def lf9(x):
    return ABSTAIN

In [ ]:
lfs = [lf0, lf1, lf2, lf3, lf4, lf5, lf6, lf7, lf8, lf9]

## (Optional) Test your function

In [ ]:
from types import SimpleNamespace

def test_func(lf, example_text):
    x = SimpleNamespace(text=example_text)
    return lf(x)

In [ ]:
test_func(lf0, "your text here")

## Applying Functions
This is how we obtain training labels, by training a model to combine the outputs of the noisy labeling functions.
`L_train` and `L_dev` are matrices representing the label returned by each labeling function for each example in the training and development sets.

In [ ]:
# Apply the LFs to the unlabeled training data, and the development data
applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)
L_dev = applier.apply(df_dev)

Train the snorkel model to combine these noisy labels.

In [ ]:
# Train the label model and compute the training labels
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)
df_train["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")
display(df_train.sample(5))

## View Unlabeled Examples
You can use these to brainstorm new labeling functions. You may try filtering or sorting them in other ways.

In [ ]:
# You can filter for unlabeled data
df_unlabeled = df_train[df_train.label == ABSTAIN]
display(df_unlabeled.sample(5))

## Analyze Results
Evaluate the accuracy of the estimated training labels and development set labels (based on ground truth).

In [ ]:
Y_train = df_train.label.values
train_analysis = LFAnalysis(L=L_train, lfs=lfs).lf_summary(Y=Y_train)
display("Training set results:", train_analysis)

In [ ]:
Y_dev = df_dev.label.values
dev_analysis = LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=Y_dev)
display("Dev set results:", dev_analysis)

## Save the Model
When you have finished.

In [ ]:
label_model.save("snorkel_amazon_lfmodel.pkl")

## Train Model
We can train a simple bag of words model on these labels to see the test accuracy.

(This step may take a while).

In [ ]:
train_model(label_model, df_train, df_valid, df_test, L_train)